In [ ]:

import csv, re, requests, os
from bs4 import BeautifulSoup
basedir = 'ipmmodels.com'
urls = ['https://www.ipmmodels.com/results/los-angeles-mainboard/all', 'https://www.ipmmodels.com/results/new-york-mainboard/all', 'https://www.ipmmodels.com/results/miami-mainboard/all']
models_details = list()


In [ ]:
soups = [BeautifulSoup(requests.get(url).content) for url in urls]

In [ ]:
models = [model['href'] for soup in soups for model in soup.find('div', attrs={'class':'results-grid'}).findAll('a')]

raw_names = [model.split('/')[-1] for model in models]
names = [" ".join(i.split('-')) for i in raw_names]
models_dict = dict(zip(names, models))
models_dict

In [ ]:
def download_pics(pics,k):
    regex = re.compile(r"\d+\.jpg")
    for pic in pics:
        file = regex.search(pic).group(0)
        path = f'{basedir}/{k}/{file}'
        if not os.path.exists(path):
            with open(path, 'wb') as f:
                f.write(requests.get(pic).content)

def list_to_csv(lst) -> None:
    with open(f"{basedir}/models_info.csv",'w') as f:
        c = csv.DictWriter(f, ["NAME","URL", 'HEIGHT', 'BUST', 'WAIST', 'HIPS','DRESS', 'SHOE', 'EYES','HAIR',"BOTTOM","TOP"])
        c.writeheader()
        for row in lst:
            c.writerow(row)


def parse(url):
    soup1 = BeautifulSoup(requests.get(url).content, 'html5lib')
    features = [stat.find('span').text.upper() for stat in soup1.find('div', attrs={'class':'portfolio-page'}).findAll('span', class_='stat')]
    values = [stat.find('span', class_='value').text for stat in soup1.find('div', attrs={'class':'portfolio-page'}).findAll('span', class_='stat')]
    return dict(zip(features, values))



In [ ]:
for k,v in models_dict.items():
    k=" ".join([i.title() for i in k.split()])
    print(k,v)
    os.makedirs(f"{basedir}/{k}", exist_ok=True)
    soup1 = BeautifulSoup(requests.get(v).content, 'html5lib')
    pics=[url.get('data-thumb') for url in soup1.find('div', attrs={'class':'portfolio-page'}).findAll('a') if url.get('data-thumb')!=None]
    feat_dict = {'NAME':k,'URL':v }
    feat_dict.update(parse(v))
    models_details.append(feat_dict)
    download_pics(pics, k)


In [ ]:

list_to_csv(models_details)